In [106]:
import torch
import numpy as np

In [107]:
x = torch.arange(2*4*5).reshape(shape=(2,4,5))
y = x[:,0::2,2:4]
print(x,y,sep='\n')

tensor([[[ 0,  1,  2,  3,  4],
         [ 5,  6,  7,  8,  9],
         [10, 11, 12, 13, 14],
         [15, 16, 17, 18, 19]],

        [[20, 21, 22, 23, 24],
         [25, 26, 27, 28, 29],
         [30, 31, 32, 33, 34],
         [35, 36, 37, 38, 39]]])
tensor([[[ 2,  3],
         [12, 13]],

        [[22, 23],
         [32, 33]]])


# Changing one view (y) also changes the original tensor (x)!

Here, `y` is slice of `x`.

We change value of `y` to 100.

Notice that `x` is also changed!

In [108]:
y[-1]=100
print(y,x,sep='\n')

tensor([[[  2,   3],
         [ 12,  13]],

        [[100, 100],
         [100, 100]]])
tensor([[[  0,   1,   2,   3,   4],
         [  5,   6,   7,   8,   9],
         [ 10,  11,  12,  13,  14],
         [ 15,  16,  17,  18,  19]],

        [[ 20,  21, 100, 100,  24],
         [ 25,  26,  27,  28,  29],
         [ 30,  31, 100, 100,  34],
         [ 35,  36,  37,  38,  39]]])


# If we don't want to change original tensor, we clone()

In [109]:
z = x[:,0::2,2:5].clone()
z[0] = 0
z, x

(tensor([[[  0,   0,   0],
          [  0,   0,   0]],
 
         [[100, 100,  24],
          [100, 100,  34]]]),
 tensor([[[  0,   1,   2,   3,   4],
          [  5,   6,   7,   8,   9],
          [ 10,  11,  12,  13,  14],
          [ 15,  16,  17,  18,  19]],
 
         [[ 20,  21, 100, 100,  24],
          [ 25,  26,  27,  28,  29],
          [ 30,  31, 100, 100,  34],
          [ 35,  36,  37,  38,  39]]]))

# When we use index, this is not slice, not a view, and the tensor is copied

In [110]:
x = torch.arange(5)
index = (x>=2) & (x<4) # index = 2, 3
y = x[index]
print(y)
y[-1]=0  # Here, X is not changed
print(x,index,y,sep='\n')

tensor([2, 3])
tensor([0, 1, 2, 3, 4])
tensor([False, False,  True,  True, False])
tensor([2, 0])


# Wasted space when want in-place operation

In [111]:
x = torch.ones(5)
address = id(x)
x = torch.exp(x)  # Here, we want to do 'in place' operation (x equals operation on x)
print(address == id(x))  # But, we can see that X has new ID, which indicating wasted space

False


## How to fix wasted space - important
Trust the operations of pytorch. 

Don't try to avoid it by messy code, trust the library.

In [112]:
address = id(x)
x[:] = torch.exp(x)  # Here, we do 'in place' operation, notice [:] at the beginning
print(address == id(x))  # We can see that 'x' is the same memory address

True


## Another way - using library '_' (underscore) functions

In [113]:
address = id(x)
x = torch.exp_(x)  # Here, notice the '_' underscore at the function name - it's in place operations
print(address == id(x))  # We can see that 'x' is the same memory address

True


# Question 1

## Way 1 - add two rows and return view

In [131]:
x = torch.arange(5)
address_before = id(x)
print(x, x.size())

x += x
print(x)

address_after = id(x)
print(address_before == address_after)

tensor([0, 1, 2, 3, 4]) torch.Size([5])
tensor([0, 2, 4, 6, 8])
True


## Way 2 - convert to column and add `x` (the correct answer)

In [143]:
x = torch.arange(5)
address_before = id(x)
print(x, x.size())

y = x.view((5,1))  # Convert to column
y += x.view((5, 1))
print(x)

address_after = id(x)
print(address_before == address_after)

tensor([0, 1, 2, 3, 4]) torch.Size([5])
tensor([0, 2, 4, 6, 8])
True
